# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('log_reg_project').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
df = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [11]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years'], outputCol='features')

In [14]:
output = assembler.transform(df)

In [15]:
final_data = output.select('features','churn')

In [17]:
train_churn, test_churn = final_data.randomSplit([0.7,0.3])

In [18]:
lr_churn = LogisticRegression(labelCol='churn')

In [19]:
fitted_churn_model = lr_churn.fit(train_churn)

In [20]:
training_sum = fitted_churn_model.summary

In [22]:
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               600|                600|
|   mean|             0.165|               0.01|
| stddev|0.3714902533677641|0.09958176311910655|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [25]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[2.36979612828959...|[0.91449492037568...|       0.0|
|[26.0,8939.61,0.0...|    0|[2.78191953646014...|[0.94169093391930...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.73773538168075...|[0.85039918836020...|       0.0|
|[28.0,11128.95,1....|    0|[2.09660446063583...|[0.89057271203825...|       0.0|
|[28.0,11204.23,0....|    0|[3.21029182114041...|[0.96121974503002...|       0.0|
|[29.0,9378.24,0.0...|    0|[2.49717687460696...|[0.92394367126366...|       0.0|
|[29.0,13255.05,1....|    0|[2.22644899451650...|[0.90259962353482...|       0.0|
|[30.0,13473.35,0....|    0|[3.10714084311229...|[0.95718633870974...|       0.0|
|[31.0,8829.83,1.0...|    0|[2.26158771697718...|[0.90564539175724...|       0.0|
|[31.0,11743.24,

In [26]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='churn')

In [27]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [28]:
auc

0.5057878573115994

In [29]:
final_lr_model = lr_churn.fit(final_data)

In [30]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [31]:
test_new_customers = assembler.transform(new_customers)

In [32]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [33]:
final_results = final_lr_model.transform(test_new_customers)

In [34]:
final_results.select("Company", 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       0.0|
|Barron-Robertson|       0.0|
|   Sexton-Golden|       0.0|
|        Wood LLC|       1.0|
|   Parks-Robbins|       0.0|
+----------------+----------+

